In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df_train = pd.read_csv("/content/drive/MyDrive/Data Science/AnalyticsVidhya-JOBATHON/train_F3fUq2S.csv")
df_train.head()

,campaign_id,sender,subject_len,body_len,mean_paragraph_len,day_of_week,is_weekend,times_of_day,category,product,...,is_image,is_personalised,is_quote,is_timer,is_emoticons,is_discount,is_price,is_urgency,target_audience,click_rate
0,1,3,76,10439,39,5,1,Noon,6,26,...,0,0,0,0,0,0,0,0,14,0.103079
1,2,3,54,2570,256,5,1,Morning,2,11,...,0,0,0,0,0,0,0,0,10,0.700000
2,3,3,59,12801,16,5,1,Noon,2,11,...,1,0,1,0,0,0,0,0,16,0.002769
3,4,3,74,11037,30,4,0,Evening,15,9,...,0,0,0,0,0,0,0,0,10,0.010868
4,5,3,80,10011,27,5,1,Noon,6,26,...,0,0,1,0,0,0,0,0,14,0.142826


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1888 entries, 0 to 1887
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   campaign_id         1888 non-null   int64  
 1   sender              1888 non-null   int64  
 2   subject_len         1888 non-null   int64  
 3   body_len            1888 non-null   int64  
 4   mean_paragraph_len  1888 non-null   int64  
 5   day_of_week         1888 non-null   int64  
 6   is_weekend          1888 non-null   int64  
 7   times_of_day        1888 non-null   object 
 8   category            1888 non-null   int64  
 9   product             1888 non-null   int64  
 10  no_of_CTA           1888 non-null   int64  
 11  mean_CTA_len        1888 non-null   int64  
 12  is_image            1888 non-null   int64  
 13  is_personalised     1888 non-null   int64  
 14  is_quote            1888 non-null   int64  
 15  is_timer            1888 non-null   int64  
 16  is_emo

Feature Engineering

Checking the data for null values

In [4]:
df_train.isnull().sum() 
#No null values are present in the dataset

campaign_id           0
sender                0
subject_len           0
body_len              0
mean_paragraph_len    0
day_of_week           0
is_weekend            0
times_of_day          0
category              0
product               0
no_of_CTA             0
mean_CTA_len          0
is_image              0
is_personalised       0
is_quote              0
is_timer              0
is_emoticons          0
is_discount           0
is_price              0
is_urgency            0
target_audience       0
click_rate            0
dtype: int64

In [5]:
#times_of_day	feature is categorical and we should encode it
day = pd.get_dummies(df_train['times_of_day'],drop_first=True)
df_train = pd.concat([df_train,day],axis=1)
df_train.drop('times_of_day',axis=1,inplace=True)
df_train.head()

,campaign_id,sender,subject_len,body_len,mean_paragraph_len,day_of_week,is_weekend,category,product,no_of_CTA,...,is_quote,is_timer,is_emoticons,is_discount,is_price,is_urgency,target_audience,click_rate,Morning,Noon
0,1,3,76,10439,39,5,1,6,26,3,...,0,0,0,0,0,0,14,0.103079,0,1
1,2,3,54,2570,256,5,1,2,11,0,...,0,0,0,0,0,0,10,0.700000,1,0
2,3,3,59,12801,16,5,1,2,11,3,...,1,0,0,0,0,0,16,0.002769,0,1
3,4,3,74,11037,30,4,0,15,9,4,...,0,0,0,0,0,0,10,0.010868,0,0
4,5,3,80,10011,27,5,1,6,26,3,...,1,0,0,0,0,0,14,0.142826,0,1


In [6]:
X = df_train.drop('click_rate',axis=1)
y = df_train['click_rate']

In [7]:
## Divide the dataset into Train and Validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

Using Random forest regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
rf.score(X_train,y_train)

0.9406809570895875

In [ ]:
rf.score(X_val,y_val)

0.43365142123058154

We can see that although the model performs pretty well on test set, it doesnot do so on validation set

We can go for Grid Search CV

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import AdaBoostRegressor
import time as time
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

In [ ]:
model_params = {
    'ADBoost': {
        'model': AdaBoostRegressor(),
        'params' : {
            'n_estimators':[50,75,100,125,150,175,200]
        }
    },
    'KNN': {
        'model':KNeighborsRegressor(),
        'params':{
            'n_neighbors':[7,9,11,13,15],
            'weights':['uniform','distance']
        }
    },
    
    'Random Forest':{
        'model':RandomForestRegressor(),
        'params' : {
            'n_estimators':[50,75,100,125,150,175,200]
        }
    },

    'SVR': {
        'model':SVR(gamma="auto"),
        'params' : {
            'C':[1,10,20],
        }
    },
  
    'ANN': {
        'model':MLPRegressor(),
        'params' : {
            'hidden_layer_sizes':[(100,),(54,108,54,27)],
            'activation':['relu'],
            'solver':['sgd','adam']
        }
    }
}

In [ ]:
scores = []

for model_name, mp in model_params.items():
    start = time.time()
    print (model_name," started...")
    clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False,scoring='r2')
    clf.fit(X_train,y_train)
    scores.append({
        'model':model_name,
        'best_score':clf.best_score_,
        'best_params':clf.best_params_
    })
    print (model_name," completed...")
    
    
    end = time.time()
    duration = end - start
    if (duration>=3600):
        hours = duration//3600
        mins = (duration%3600)//60
        secs = duration%60
    elif (duration>=60):
        hours = 0
        mins= duration//60
        secs = duration%60
    else:
        hours = 0
        mins = 0
        secs = duration
    print (round(hours,2), " hr(s) ",round(mins,2)," min(s) ",round(secs,2), " sec(s)\n")

ADBoost  started...
ADBoost  completed...
0  hr(s)  0  min(s)  9.66  sec(s)

KNN  started...
KNN  completed...
0  hr(s)  0  min(s)  0.82  sec(s)

Random Forest  started...
Random Forest  completed...
0  hr(s)  0  min(s)  38.48  sec(s)

SVR  started...
SVR  completed...
0  hr(s)  0  min(s)  1.5  sec(s)

ANN  started...
ANN  completed...
0  hr(s)  0  min(s)  35.72  sec(s)



In [ ]:
results_df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
results_df.sort_values('best_score',ascending=False)

,model,best_score,best_params
2,Random Forest,0.497358,{'n_estimators': 75}
1,KNN,0.338419,"{'n_neighbors': 15, 'weights': 'distance'}"
0,ADBoost,0.024644,{'n_estimators': 175}
3,SVR,-0.855046,{'C': 1}
4,ANN,-1218.251939,"{'activation': 'relu', 'hidden_layer_sizes': (..."


Training the RF model with best params on the entire train set

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RFfinal = RandomForestRegressor(n_estimators=75)
RFfinal.fit(X, y)
RFfinal.score(X,y)

0.9391590500254056

Testing on test.csv

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/Data Science/AnalyticsVidhya-JOBATHON/test_Bk2wfZ3.csv')
df_test.head()

,campaign_id,sender,subject_len,body_len,mean_paragraph_len,day_of_week,is_weekend,times_of_day,category,product,...,mean_CTA_len,is_image,is_personalised,is_quote,is_timer,is_emoticons,is_discount,is_price,is_urgency,target_audience
0,1889,3,61,12871,11,6,1,Noon,6,26,...,37,1,0,0,0,1,0,0,0,14
1,1890,3,54,2569,256,5,1,Morning,2,11,...,22,0,0,0,0,0,0,0,0,10
2,1891,3,88,1473,78,4,0,Evening,2,11,...,22,1,0,1,0,0,0,0,0,10
3,1892,3,88,1473,78,3,0,Evening,2,11,...,22,1,0,1,0,0,0,0,0,10
4,1893,3,78,9020,29,3,0,Noon,6,26,...,40,0,0,0,0,0,0,0,0,14


In [ ]:
#times_of_day	feature is categorical and we should encode it
day = pd.get_dummies(df_test['times_of_day'],drop_first=True)
df_test = pd.concat([df_test,day],axis=1)
df_test.drop('times_of_day',axis=1,inplace=True)
df_test.head()

,campaign_id,sender,subject_len,body_len,mean_paragraph_len,day_of_week,is_weekend,category,product,no_of_CTA,...,is_personalised,is_quote,is_timer,is_emoticons,is_discount,is_price,is_urgency,target_audience,Morning,Noon
0,1889,3,61,12871,11,6,1,6,26,3,...,0,0,0,1,0,0,0,14,0,1
1,1890,3,54,2569,256,5,1,2,11,0,...,0,0,0,0,0,0,0,10,1,0
2,1891,3,88,1473,78,4,0,2,11,0,...,0,1,0,0,0,0,0,10,0,0
3,1892,3,88,1473,78,3,0,2,11,0,...,0,1,0,0,0,0,0,10,0,0
4,1893,3,78,9020,29,3,0,6,26,2,...,0,0,0,0,0,0,0,14,0,1


In [ ]:
X = df_test
df_test['click_rate'] = RFfinal.predict(X)

In [ ]:
df_test

,campaign_id,sender,subject_len,body_len,mean_paragraph_len,day_of_week,is_weekend,category,product,no_of_CTA,...,is_quote,is_timer,is_emoticons,is_discount,is_price,is_urgency,target_audience,Morning,Noon,click_rate
0,1889,3,61,12871,11,6,1,6,26,3,...,0,0,1,0,0,0,14,0,1,0.094750
1,1890,3,54,2569,256,5,1,2,11,0,...,0,0,0,0,0,0,10,1,0,0.334363
2,1891,3,88,1473,78,4,0,2,11,0,...,1,0,0,0,0,0,10,0,0,0.271743
3,1892,3,88,1473,78,3,0,2,11,0,...,1,0,0,0,0,0,10,0,0,0.280960
4,1893,3,78,9020,29,3,0,6,26,2,...,0,0,0,0,0,0,14,0,1,0.126838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,2646,3,72,9081,46,0,0,15,9,4,...,0,0,0,0,0,0,7,0,0,0.028227
758,2647,3,58,10537,40,3,0,2,11,5,...,0,0,0,0,0,0,16,0,0,0.024202
759,2648,3,100,9132,19,2,0,15,9,2,...,0,0,0,0,0,0,15,0,0,0.047264
760,2649,3,88,1451,75,1,0,2,11,0,...,1,0,0,0,0,0,10,0,1,0.295580


In [ ]:
sub_df = df_test[['campaign_id','click_rate']]
sub_df

,campaign_id,click_rate
0,1889,0.094750
1,1890,0.334363
2,1891,0.271743
3,1892,0.280960
4,1893,0.126838
...,...,...
757,2646,0.028227
758,2647,0.024202
759,2648,0.047264
760,2649,0.295580


In [ ]:
sub_df.to_csv('submission.csv',index=False)#score=0.43

Trying other models and methods as this score was no that high

In [ ]:
# create an xgboost regression model
XGBmodel = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)
XGBmodel.fit(X, y)
XGBmodel.score(X,y)

[09:40:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.9999409230395832

In [ ]:
xgbpred = XGBmodel.predict(df_test.drop('click_rate',axis=1))

In [ ]:
df_test['click_rate'] = xgbpred
sub_df2 = df_test[['campaign_id','click_rate']]
sub_df2

,campaign_id,click_rate
0,1889,0.118048
1,1890,0.406780
2,1891,0.288755
3,1892,0.287173
4,1893,0.137989
...,...,...
757,2646,0.015006
758,2647,0.003825
759,2648,0.006545
760,2649,0.352463


In [ ]:
sub_df2.to_csv('submission2.csv',index=False) # score 0.53

Tinkering with XGBOOST

In [ ]:
# create an xgboost regression model
# Various hyper-parameters to tune
xgb1 = XGBRegressor()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7,8,9,10],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500,1000,1500]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X,y)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 2 folds for each of 54 candidates, totalling 108 fits
-0.0724473267059067
{'colsample_bytree': 0.7, 'learning_rate': 0.03, 'max_depth': 7, 'min_child_weight': 4, 'n_estimators': 500, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}


In [ ]:
XGBmodel = XGBRegressor(colsample_bytree= 0.7, learning_rate= 0.03, max_depth=7, min_child_weight=4, n_estimators= 1500,subsample= 0.7,objective = 'reg:squarederror')


In [ ]:
XGBmodel.fit(X,y)
XGBmodel.score(X,y)

0.9971420724638812

In [ ]:
X = df_test
df_test['click_rate'] = XGBmodel.predict(X)

In [ ]:
sub_df3 = df_test[['campaign_id','click_rate']]
sub_df3

,campaign_id,click_rate
0,1889,0.109953
1,1890,0.370545
2,1891,0.213321
3,1892,0.236799
4,1893,0.144192
...,...,...
757,2646,0.019295
758,2647,0.008124
759,2648,0.032448
760,2649,0.317466


In [ ]:
sub_df3.to_csv('Submission-3.csv',index=False)#0.56

Another XGBOOST

In [ ]:
XGBmodel3 = XGBRegressor(colsample_bytree= 0.7, learning_rate= 0.1, max_depth=10, min_child_weight=4, n_estimators= 1500,subsample= 0.7,objective = 'reg:squarederror')
XGBmodel3.fit(X,y)
XGBmodel3.score(X,y)

0.9999451756319652

In [ ]:
X2 = df_test.drop('click_rate',axis=1)
df_test['click_rate'] = XGBmodel3.predict(X2)

In [ ]:
sub_df4 = df_test[['campaign_id','click_rate']]
sub_df4

,campaign_id,click_rate
0,1889,0.098727
1,1890,0.419278
2,1891,0.179355
3,1892,0.207092
4,1893,0.140198
...,...,...
757,2646,0.020221
758,2647,0.004469
759,2648,0.042205
760,2649,0.342968


In [ ]:
sub_df4.to_csv('Submission-4.csv',index=False)#0.52

Trying Feature selection to improve results

In [8]:
from sklearn.feature_selection import mutual_info_regression
# determine the mutual information
mutual_info = mutual_info_regression(X_train, y_train)
mutual_info

array([0.06907023, 0.0181298 , 0.06213984, 0.03047786, 0.02436152,
       0.03291319, 0.02606955, 0.03821652, 0.02506273, 0.01868914,
       0.04817624, 0.02614186, 0.02478711, 0.01477622, 0.0002566 ,
       0.03582226, 0.01654068, 0.0010287 , 0.02757817, 0.06060829,
       0.00727073, 0.00369519])

In [9]:
mutual_info = pd.Series(mutual_info)
mutual_info.index = X_train.columns
mutual_info.sort_values(ascending=False)

campaign_id           0.069070
subject_len           0.062140
target_audience       0.060608
mean_CTA_len          0.048176
category              0.038217
is_emoticons          0.035822
day_of_week           0.032913
body_len              0.030478
is_urgency            0.027578
is_image              0.026142
is_weekend            0.026070
product               0.025063
is_personalised       0.024787
mean_paragraph_len    0.024362
no_of_CTA             0.018689
sender                0.018130
is_discount           0.016541
is_quote              0.014776
Morning               0.007271
Noon                  0.003695
is_price              0.001029
is_timer              0.000257
dtype: float64

In [12]:
from sklearn.feature_selection import SelectKBest
#select the  top 10 important features
cols = SelectKBest(mutual_info_regression, k=10)
cols.fit(X_train, y_train)
X_train.columns[cols.get_support()]

Index(['campaign_id', 'subject_len', 'body_len', 'day_of_week', 'category',
       'mean_CTA_len', 'is_personalised', 'is_emoticons', 'is_urgency',
       'target_audience'],
      dtype='object')

In [16]:
X_train_10Best = X_train[X_train.columns[cols.get_support()]]
X_val_10Best = X_val[X_train.columns[cols.get_support()]]

In [28]:
XGBmodel4_FS = XGBRegressor(colsample_bytree= 0.7, learning_rate= 0.03, max_depth=7, min_child_weight=4, n_estimators= 1500,subsample= 0.7,objective = 'reg:squarederror')
score = []
for i in range(1,21):
  cols = SelectKBest(mutual_info_regression, k=i)
  cols.fit(X_train, y_train)
  X_train.columns[cols.get_support()]
  X_train_10Best = X_train[X_train.columns[cols.get_support()]]
  X_val_10Best = X_val[X_train.columns[cols.get_support()]]
  XGBmodel4_FS.fit(X_train_10Best,y_train)
  score.append(XGBmodel4_FS.score(X_val_10Best,y_val))

In [29]:
score

[-0.4578825678824887,
 -0.21106409701994489,
 -0.16940278461359082,
 0.03150672944685562,
 0.15540829783383003,
 0.14891104868073257,
 0.11047379931404733,
 0.14075811418974216,
 0.15721263730238266,
 0.4268697503406107,
 0.4000397496876359,
 0.41385011136080496,
 0.4167639896882497,
 0.40137670034164297,
 0.4426768309603988,
 0.4378567707707218,
 0.42243694725132264,
 0.45265589137288753,
 0.4573060644304682,
 0.4475698364792766]

In [27]:
np.argmax(score) #therefore for k=13, the model gives best results

18

In [23]:
cols = SelectKBest(mutual_info_regression, k=13)
cols.fit(X_train, y_train)
X_train.columns[cols.get_support()]
X_train_10Best = X_train[X_train.columns[cols.get_support()]]
X_val_10Best = X_val[X_train.columns[cols.get_support()]]
XGBmodel4_FS.fit(X_train_10Best,y_train)
XGBmodel4_FS.score(X_val_10Best,y_val)

0.4167639896882497